In [1]:
from notes.strategy import XOverStrat, Benchmark
from ib_insync import *
import scripts.trend_viewer
util.startLoop()

ib = IB()
# port = 7496
port = 4001
ib.connect('127.0.0.1', port, clientId=0)

<IB connected to 127.0.0.1:4001 clientId=0>

In [2]:
def ib_get_historical(symbol, sec_type, duration, interval, use_rth=True): 
    contracts = ib.reqContractDetails(Contract(symbol=symbol, secType=sec_type, includeExpired=False))[0]
    contract = contracts.contract
    bars = ib.reqHistoricalData(
        contract,
        endDateTime='',
        durationStr=duration,
        barSizeSetting=interval,
        whatToShow='TRADES' if symbol not in ['BTC', 'ETH'] else 'AGGTRADES',
        useRTH=use_rth,
        formatDate=1,
        keepUpToDate=False
    )
    prices = util.df(bars)
    prices.index = prices.date 
    prices = prices.drop(columns=['date'])
    return prices


In [3]:
price = ib_get_historical('BTC', 'CRYPTO', '5 Y', '1 day')

In [4]:
bench = Benchmark()
bench.update(price['close'])

In [5]:
bench.stats.expectancy()

date
2019-04-08         NaN
2019-04-09    0.001200
2019-04-10    0.012702
2019-04-11   -0.004153
2019-04-12   -0.006103
                ...   
2024-04-01    0.001704
2024-04-02    0.001672
2024-04-03    0.001658
2024-04-04    0.001683
2024-04-05    0.001667
Name: close, Length: 1305, dtype: float64